In [48]:
import pandas as pd
import sqlite3
import datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

from sqlalchemy import create_engine

In [49]:
#connection to a PostgreSQL local database
# ===========================================

# user_name_var = input('What is the username of your database? ')
host_var = input('What is the host address of your database?')
pw = input("Enter the database password: ")
# db_name = input("Enter the name of the database: ")

string_output = "postgresql://postgres"+':'+pw+'@'+host_var+":5432"+'/earthquake_db'
engine = create_engine(string_output)
connection = engine.connect()
print("Connected to database...")

Connected to database...


In [50]:
#create and connect to a sqlite3 db
conn = sqlite3.connect(r'/Users/rosaicelaroman/Desktop/portfolio_work/earthquake_modeling/Database/db/earthquake_db')

Adding 1 month of data

In [51]:
page_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"

df = pd.read_csv(page_url)
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-03-31T21:00:50.670Z,38.821499,-122.793167,3.21,0.35,md,10.0,90.0,0.012760,0.01,...,2022-03-31T21:02:26.715Z,"6km NNW of The Geysers, CA",earthquake,0.63,1.10,NaN,1.0,automatic,nc,nc
1,2022-03-31T20:10:49.140Z,36.184500,-118.008333,3.91,0.78,ml,18.0,87.0,0.085760,0.12,...,2022-03-31T20:39:06.832Z,"11km S of Olancha, CA",earthquake,0.25,0.57,0.085,5.0,reviewed,ci,ci
2,2022-03-31T20:06:46.933Z,60.899400,-152.448700,12.40,0.80,ml,NaN,NaN,NaN,0.35,...,2022-03-31T20:15:24.697Z,"67 km WNW of Nikiski, Alaska",earthquake,NaN,0.40,NaN,NaN,automatic,ak,ak
3,2022-03-31T20:05:10.760Z,37.409500,-118.599333,11.03,2.21,md,21.0,71.0,0.063150,0.04,...,2022-03-31T20:47:11.513Z,"1km SSW of Round Valley, CA",earthquake,0.45,1.41,0.257,25.0,reviewed,nc,nc
4,2022-03-31T19:58:41.070Z,38.806500,-122.790832,0.62,0.73,md,17.0,48.0,0.003788,0.03,...,2022-03-31T20:16:15.311Z,"4km NW of The Geysers, CA",earthquake,0.20,0.40,0.240,3.0,automatic,nc,nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9385,2022-03-01T21:41:08.610Z,32.898667,-115.519500,7.93,1.60,ml,24.0,86.0,0.034780,0.18,...,2022-03-03T16:26:06.430Z,"7km NE of Imperial, CA",earthquake,0.27,0.26,0.147,14.0,reviewed,ci,ci
9386,2022-03-01T21:27:08.956Z,58.286600,-154.854200,0.00,2.30,ml,NaN,NaN,NaN,1.06,...,2022-03-01T21:38:34.240Z,"83 km NNW of Karluk, Alaska",earthquake,NaN,0.30,NaN,NaN,automatic,ak,ak
9387,2022-03-01T21:22:14.615Z,60.952300,-151.257700,0.00,2.20,ml,NaN,NaN,NaN,0.91,...,2022-03-01T21:26:42.710Z,"14 km SSW of Tyonek, Alaska",earthquake,NaN,0.50,NaN,NaN,automatic,ak,ak
9388,2022-03-01T21:16:45.780Z,39.425333,-110.319667,-1.21,1.42,md,5.0,192.0,0.004385,0.04,...,2022-03-01T22:46:44.500Z,"15 km SSE of Sunnyside, Utah",earthquake,1.33,0.22,0.212,4.0,reviewed,uu,uu


In [52]:
#append the new data from USGS to both databases
#=============
#PostgreSQL
df.to_sql('earthquake_raw', engine, if_exists='append', index = False)
#=============
#SQLite
cur = conn.cursor()
df.to_sql('earthquake_raw', conn, if_exists='append', index=False)

9390

Load from database to update with new rows and remove duplicate rows

In [53]:
#Dataframe from postgresql
model_df = pd.read_sql_table(
    "earthquake_raw",
    con=engine
)
model_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.875000,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.119000,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48063,2022-03-01T21:41:08.610Z,32.898667,-115.519500,7.93,1.60,ml,24.0,86.00,0.034780,0.1800,...,2022-03-03T16:26:06.430Z,"7km NE of Imperial, CA",earthquake,0.27,0.26,0.147,14.0,reviewed,ci,ci
48064,2022-03-01T21:27:08.956Z,58.286600,-154.854200,0.00,2.30,ml,NaN,NaN,NaN,1.0600,...,2022-03-01T21:38:34.240Z,"83 km NNW of Karluk, Alaska",earthquake,NaN,0.30,NaN,NaN,automatic,ak,ak
48065,2022-03-01T21:22:14.615Z,60.952300,-151.257700,0.00,2.20,ml,NaN,NaN,NaN,0.9100,...,2022-03-01T21:26:42.710Z,"14 km SSW of Tyonek, Alaska",earthquake,NaN,0.50,NaN,NaN,automatic,ak,ak
48066,2022-03-01T21:16:45.780Z,39.425333,-110.319667,-1.21,1.42,md,5.0,192.00,0.004385,0.0400,...,2022-03-01T22:46:44.500Z,"15 km SSE of Sunnyside, Utah",earthquake,1.33,0.22,0.212,4.0,reviewed,uu,uu


In [54]:
#dataframe from SQLite
model_df2 = pd.read_sql_query("SELECT * FROM earthquake_raw", conn)
model_df2

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.875000,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.119000,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48063,2022-03-01T21:41:08.610Z,32.898667,-115.519500,7.93,1.60,ml,24.0,86.00,0.034780,0.1800,...,2022-03-03T16:26:06.430Z,"7km NE of Imperial, CA",earthquake,0.27,0.26,0.147,14.0,reviewed,ci,ci
48064,2022-03-01T21:27:08.956Z,58.286600,-154.854200,0.00,2.30,ml,NaN,NaN,NaN,1.0600,...,2022-03-01T21:38:34.240Z,"83 km NNW of Karluk, Alaska",earthquake,NaN,0.30,NaN,NaN,automatic,ak,ak
48065,2022-03-01T21:22:14.615Z,60.952300,-151.257700,0.00,2.20,ml,NaN,NaN,NaN,0.9100,...,2022-03-01T21:26:42.710Z,"14 km SSW of Tyonek, Alaska",earthquake,NaN,0.50,NaN,NaN,automatic,ak,ak
48066,2022-03-01T21:16:45.780Z,39.425333,-110.319667,-1.21,1.42,md,5.0,192.00,0.004385,0.0400,...,2022-03-01T22:46:44.500Z,"15 km SSE of Sunnyside, Utah",earthquake,1.33,0.22,0.212,4.0,reviewed,uu,uu


In [56]:
#check for duplicates in postgres data
boolean = model_df.duplicated(subset=['id']).any()
print(boolean)

True


In [57]:
#check for duplicates in sqlite data
boolean = model_df2.duplicated(subset=['id']).any()
print(boolean)

True


In [58]:
#drop duplicates in postgres data
model_df.drop_duplicates(subset=['id'], inplace=True)
print(model_df)

                           time   latitude   longitude   depth   mag magType  \
0      2022-01-06T01:42:39.855Z  37.384000 -117.107500    1.10  1.30      ml   
1      2022-01-06T01:31:33.698Z  38.181000 -117.818000    7.30  1.40      ml   
2      2022-01-06T01:24:20.083Z  61.340700 -149.885200   30.00  1.50      ml   
3      2022-01-06T01:15:15.851Z  62.870300 -150.762500  111.30  2.10      ml   
4      2022-01-06T01:10:44.629Z  64.014500 -148.765700    1.90  1.50      ml   
...                         ...        ...         ...     ...   ...     ...   
38682  2022-03-03T05:35:27.488Z  44.465000 -115.206100    6.34  2.20      ml   
38683  2022-03-02T14:20:15.918Z -20.692700 -175.521600   10.00  4.80      mb   
38684  2022-03-02T13:09:03.072Z -30.286900 -177.465000   10.00  4.30      mb   
38685  2022-03-31T21:00:50.670Z  38.821499 -122.793167    3.21  0.35      md   
40392  2022-03-26T01:25:27.831Z -20.535000 -177.562400  490.64  4.40      mb   

        nst     gap     dmin     rms  .

In [59]:
#drop duplicates in sqlite data
model_df2.drop_duplicates(subset=['id'], inplace=True)
print(model_df2)

                           time   latitude   longitude   depth   mag magType  \
0      2022-01-06T01:42:39.855Z  37.384000 -117.107500    1.10  1.30      ml   
1      2022-01-06T01:31:33.698Z  38.181000 -117.818000    7.30  1.40      ml   
2      2022-01-06T01:24:20.083Z  61.340700 -149.885200   30.00  1.50      ml   
3      2022-01-06T01:15:15.851Z  62.870300 -150.762500  111.30  2.10      ml   
4      2022-01-06T01:10:44.629Z  64.014500 -148.765700    1.90  1.50      ml   
...                         ...        ...         ...     ...   ...     ...   
38675  2022-03-01T21:22:14.615Z  60.952300 -151.257700    0.00  2.20      ml   
38676  2022-03-01T21:16:45.780Z  39.425333 -110.319667   -1.21  1.42      md   
38677  2022-03-01T21:08:58.910Z  38.835167 -122.800003    2.22  0.85      md   
38678  2022-03-31T21:00:50.670Z  38.821499 -122.793167    3.21  0.35      md   
40384  2022-03-26T01:25:27.831Z -20.535000 -177.562400  490.64  4.40      mb   

        nst     gap      dmin     rms  

In [60]:
#replace the table in postgres with one without duplicates
model_df.to_sql('earthquake_raw', engine, if_exists='replace', index = False)

680

In [61]:
#replace the table in sqlite with one without duplicates
model_df2.to_sql('earthquake_raw', conn, if_exists='replace', index = False)

38680

In [62]:
#copy data as csv as backup method
model_df.to_csv("../Database/csv/earthquake_raw.csv",index=False)

In [63]:
# Close connection to SQLite database
conn.close()
connection.close()